In [2]:
%pip install -q petals
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import BloomTokenizerFast 
from petals import DistributedBloomForCausalLM
import numpy as np

MODEL_NAME = "bigscience/bloom-petals"
tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME)
model = model.cuda()

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.19G [00:00<?, ?B/s]

In [4]:
import jsonlines

question,passage,answer=[],[],[]
with jsonlines.open('train.jsonl') as f:
    for line in f.iter():
        question.append(line['question']) # or whatever else you'd like to do
        passage.append(line['passage'])
        answer.append(line['answer'])

In [5]:
prompt="passage: \"The tower's tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure's weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries.\"\nquestion: \"was the leaning tower of pisa built leaning\"\nanswer: false\n\npassage: \"In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901.\"\nquestion: \"does each australian state have its own constitution\"\nanswer: true\n\npassage: \"Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be official when the Senate receives a signed nomination letter from the president naming the nominee, which is then entered in the Senate's record. There have been 37 unsuccessful nominations to the Supreme Court of the United States. Of these, 11 nominees were rejected in Senate roll-call votes, 11 were withdrawn by the president, and 15 lapsed at the end of a session of Congress. Six of these unsuccessful nominees were subsequently nominated and confirmed to other seats on the Court. Additionally, although confirmed, seven nominees either declined office or (in one instance) died before assuming office.\"\nquestion: \"has any supreme court nominee not been confirmed\"\nanswer: true\n\npassage: \"This list contains the top 50 accounts with the most followers on the social photo-sharing platform Instagram. As of October 2018, the most followed user is Instagram's own account, with over 260 million followers. Cristiano Ronaldo is the most followed individual, with over 144 million followers. Twelve accounts have exceeded 100 million followers on the site.\"\nquestion: \"does anyone on instagram have 1 billion followers\"\nanswer: false\n\npassage: \"Creme eggs are available annually between 1 January and Easter Day. In the UK in the 1980s, Cadbury made Creme Eggs available year-round but sales dropped and they returned to seasonal availability.\"\nquestion: \"can you buy cadburys creme eggs all year round\"\nanswer: false\n\npassage: \"Station 19 is an American action-drama television series created by Stacy McKee for ABC. McKee, Shonda Rhimes, Betsy Beers, and Paris Barclay serve as executive producers on the series, which is the second spin-off to Grey's Anatomy. Set in Seattle, the series focuses on the lives of the men and women at Seattle Fire Station 19. The series is produced by Shondaland and ABC Studios, with McKee serving as showrunner.\"\nquestion: \"is station 19 a grey's anatomy spin off\"\nanswer: true"

In [16]:
pmt = "\n\npassage: " + passage[0] + "\nquestion: " + question[0] + "\nanswer: "
prompt + pmt

'passage: "The tower\'s tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure\'s weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries."\nquestion: "was the leaning tower of pisa built leaning"\nanswer: false\n\npassage: "In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901."\nquestion: "does each australian state have its own constitution"\nanswer: true\n\npassage: "Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be officia

In [18]:
inputs = tokenizer(prompt+pmt, return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=3)
test = tokenizer.decode(outputs[0])

In [37]:
print(test[-10:-6])

true


In [38]:
output = []
for i in range(30,60):
  pmt = "\n\npassage: " + passage[i] + "\nquestion: " + question[i] + "\nanswer: "
  inputs = tokenizer(prompt+pmt, return_tensors="pt")["input_ids"].cuda()
  response = model.generate(inputs, max_new_tokens=3)
  output.append(response)

In [78]:
print(tokenizer.decode(output[1][0]))

passage: "The tower's tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure's weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries."
question: "was the leaning tower of pisa built leaning"
answer: false

passage: "In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901."
question: "does each australian state have its own constitution"
answer: true

passage: "Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be official when the 

In [75]:
predict = []
pred = [tokenizer.decode(i[0]) for i in output]
for i in pred:
  predict.append(i[-10:-6])

In [76]:
prediction = [predict[i] in ('true') for i in range(30)]

In [77]:
validation = np.array(answer[30:60]) == np.array(prediction)
validation.sum() / len(validation)

0.7333333333333333

## Extra Credit 7: Fine-tune Petals on 50 random instances from BoolQ

If you'd like to talk to the model in an interactive way, you can use the __inference session__ interface. This interface provides a simple way to print generated tokens on the fly or make a chat bot that responds to human's phrases.

The inference session looks for a sequence of servers that will run successive inference steps and store past attention caches. This way, you don't need to rerun previous tokens through the transformer to generate each phrase. If one of the remote servers fails, Petals will automatically find a replacement and regenerate a small part of the caches.

Let's see how to use it to write a simple chat bot, showing tokens as soon as they are generated:

In [6]:
finetune_prompt = ""
for i in range(70,120):
  pmt = "\n\npassage: " + passage[i] + "\nquestion: " + question[i] + "\nanswer: " + str(answer[0])
  finetune_prompt = finetune_prompt + pmt

In [ ]:
print(finetune_prompt+test)

In [ ]:
output_finetune = []
for i in range(30,60):
  pmt = "\n\npassage: " + passage[i] + "\nquestion: " + question[i] + "\nanswer: "
  inputs = tokenizer(finetune_prompt+pmt, return_tensors="pt")["input_ids"].cuda()
  response = model.generate(inputs, max_new_tokens=3)
  output_finetune.append(tokenizer.decode(response[0]))

Apr 01 06:29:13.484 [INFO] Peer 12D3KooWBXHCDCerCREpypXbJ93Hudcm9bU5KWrxLajRJSvYHZqF did not respond, banning it temporarily
Apr 01 06:29:13.486 [WARN] [petals.client.inference_session.step:313] Caught exception when running inference via RemoteSpanInfo(start=0, end=27, peer_id=<libp2p.peer.id.ID (12D3KooWBXHCDCerCREpypXbJ93Hudcm9bU5KWrxLajRJSvYHZqF)>) (retry in 0 sec): P2PHandlerError('Failed to call handler `TransformerConnectionHandler.rpc_inference` at 12D3KooWBXHCDCerCREpypXbJ93Hudcm9bU5KWrxLajRJSvYHZqF: Cannot allocate KV cache for 6791 tokens, max = 2048')
Apr 01 06:29:13.493 [WARN] [petals.client.routing.sequence_manager.make_sequence:121] Remote SequenceManager is still searching for routes, waiting for it to become ready
Apr 01 06:29:39.260 [INFO] Peer 12D3KooWBXHCDCerCREpypXbJ93Hudcm9bU5KWrxLajRJSvYHZqF did not respond, banning it temporarily
Apr 01 06:29:39.262 [WARN] [petals.client.inference_session.step:313] Caught exception when running inference via RemoteSpanInfo(start

In [21]:
print(output_finetune[0])

passage: "The tower's tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure's weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries."
question: "was the leaning tower of pisa built leaning"
answer: false

passage: "In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901."
question: "does each australian state have its own constitution"
answer: true

passage: "Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be official when the 

In [19]:
pred_finetune = []
for i in output_finetune:
  pred_finetune.append(i[-10:-6])
prediction_finetune = [pred_finetune[i] in ('true') for i in range(30)]
validation_finetune = np.array(answer[30:60]) == np.array(prediction_finetune)
validation_finetune.sum() / len(validation_finetune)

0.7333333333333333